In [ ]:
# 1. Load dependencies and libraries
!pip install llama-index ragstack-ai pdfplumber transformers optimum[exporters] llama-index-llms-groq llama-index-embeddings-huggingface-optimum

import os
from getpass import getpass
import pdfplumber
from llama_index.core import Document, StorageContext, VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.vector_stores.astra_db import AstraDBVectorStore
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from llama_index.llms.groq import Groq


In [ ]:
os.environ["ASTRA_DB_API_ENDPOINT"] = "https://7c7e7cda-bb99-491a-88a3-dd75d41c0b0d-us-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = "AstraCS:PEZqYNpmWDhAynggLjtTvqpM:ad3b93fe4f7c9609ec21668984073da866093171657e4611f4ccc88602235bcf"
os.environ["GROQ_API_KEY"] = "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"

Copy the cell 2 from previous code and paste in above cell

In [ ]:
llm = Groq(model="mixtral-8x7b-32768")
Settings.llm = llm


In [ ]:
# 2. Load a PDF and extract text
pdf_path = "/content/agenda.pdf"  # Replace with your actual PDF path

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_text = extract_text_from_pdf(pdf_path)

# New function to chunk text
def chunk_text(text, chunk_size=250):  # Reduced chunk size
    """Split text into smaller chunks of specified size."""
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# Chunk the extracted PDF text
text_chunks = chunk_text(pdf_text)

# Check if we have enough chunks
if len(text_chunks) < 10:
    print("Warning: Not enough chunks created. Consider reducing the chunk size further or checking the PDF content.")

print(f"Number of chunks created: {len(text_chunks)}")
for idx, chunk in enumerate(text_chunks):
    print(f"Chunk {idx + 1} ({len(chunk)} characters):\n{chunk}\n")

# Create Document objects for each chunk
documents = [Document(text=chunk) for chunk in text_chunks]

Number of chunks created: 36
Chunk 1 (250 characters):
Agenda
Global IndiaAI Summit 2024 3-4 July 2024, New Delhi
Day 1 - Wednesday, July 3
Start Session Description
10:00 - 11:00
Opening Ceremony (Auditorium 1)
11:00 – 13:30
GPAI Session (@ Summit Room) : Executive Council Meeting
(By Invitation only)
1

Chunk 2 (250 characters):
1:30 - 13:00 Side Event 1 (@ The session is about learning and sharing knowledge on Large
Auditorium 1): Language Models (LLMs) and Large Multimodal Models (LMMs),
with the aim to understand the unique linguistic and cultural
IndiaAI: Large diversity

Chunk 3 (250 characters):
 inherent to India, examining how LLMs can effectively
Language Model address challenges associated with multilingualism. Furthermore,
the session will cover the ethical considerations and biases linked
to these models, promoting a discussion on resp

Chunk 4 (250 characters):
onsible AI
practices, including fairness, inclusiveness, misinformation
mitigation, and intellectual property r

In [ ]:

# 3. Set up the embedding model
OptimumEmbedding.create_and_save_optimum_model(
    "BAAI/bge-small-en-v1.5", "./bge_onnx"
)

embed_model = OptimumEmbedding(folder_name="./bge_onnx")

# Check embedding dimension
test_embedding = embed_model.get_text_embedding("Test sentence")
print(f"Embedding dimension: {len(test_embedding)}")

Settings.embed_model = embed_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: BertModel *****
Using framework PyTorch: 2.2.1+cu121
Overriding 1 configuration item(s)
	- use_cache -> False


Saved optimum model to ./bge_onnx. Use it with `embed_model = OptimumEmbedding(folder_name='./bge_onnx')`.
Embedding dimension: 384


In [ ]:
import uuid # Import the uuid module

# 4. Send data to vector store and store as embedding
# Generate a unique collection name
collection_name = f"pdf_embeddings_{uuid.uuid4().hex[:8]}"

astra_db_store = AstraDBVectorStore(
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
    collection_name=collection_name,
    embedding_dimension=len(test_embedding),
)

storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

# Store all documents in the vector store
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)

Parsing nodes:   0%|          | 0/36 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
# 5. Query the data
query_engine = index.as_query_engine()

# Example query
query_string_1 = "summary of event?"
response = query_engine.query(query_string_1)
print(query_string_1)
print(response.response)

# Retrieve nodes with scores
retriever = index.as_retriever(
    vector_store_query_mode="default",
    similarity_top_k=10,
)

nodes_with_scores = retriever.retrieve(query_string_1)
print(f"\nFound {len(nodes_with_scores)} nodes.")
for idx, node_with_score in enumerate(nodes_with_scores):
    print(f" [{idx}] score = {node_with_score.score:.4f}")
    print(f" text = {node_with_score.node.text[:180]} ...")

NameError: name 'index' is not defined

In [ ]:
# 6. Contextual Querying
def contextual_query(query, context, query_engine):
    """Perform a query with added context."""
    # Combine context with the query
    combined_query = f"{context}\n{query}"
    response = query_engine.query(combined_query)
    return response

# Define context for the query
context = "This document discusses various events and their summaries."

# Perform the contextual query
query_string_1 = "summary of event?"
response_with_context = contextual_query(query_string_1, context, query_engine)

# Perform the query without context
response_without_context = query_engine.query(query_string_1)

# Retrieve nodes with scores for the contextual query
nodes_with_scores_context = retriever.retrieve(query_string_1)

# Display the results
print("Response with Context:")
print(response_with_context.response)

print("\nTop 10 Nodes with Context:")
for idx, node_with_score in enumerate(nodes_with_scores_context[:10]):
    print(f" [{idx + 1}] score = {node_with_score.score:.4f}")
    print(f" text = {node_with_score.node.text[:180]} ...")

# Perform the query without context and retrieve nodes
nodes_with_scores_no_context = retriever.retrieve(query_string_1)

print("\nResponse without Context:")
print(response_without_context.response)

print("\nTop 10 Nodes without Context:")
for idx, node_with_score in enumerate(nodes_with_scores_no_context[:10]):
    print(f" [{idx + 1}] score = {node_with_score.score:.4f}")
    print(f" text = {node_with_score.node.text[:180]} ...")

Response with Context:
Sure, I'd be happy to help summarize an event from the provided context. 

One of the events is a side event taking place in Large Auditorium 1 from 1:30 to 13:00. This event is focused on Large Language Models (LLMs) and Large Multimodal Models (LMMs). The aim of this session is to facilitate learning and sharing of knowledge related to the unique linguistic and cultural diversity that these models can handle. 

I hope this summary is helpful! If you have any other questions, feel free to ask.

Top 10 Nodes with Context:
 [1] score = 0.8583
 text = 1:30 - 13:00 Side Event 1 (@ The session is about learning and sharing knowledge on Large
Auditorium 1): Language Models (LLMs) and Large Multimodal Models (LMMs),
with the aim to  ...
 [2] score = 0.8538
 text = on (@ Summit Room) : Ministerial Council Meeting
(By Invitation only)
14.30 -16:00 Side Event 3 (@ The case study presentations and panel discussion will bring
Auditorium 1): toget ...
 [3] score = 0.8393
 te

In [ ]:
# 1. Generate Initial Response
def generate_response(query, query_engine):
    """Generate a response based on the user's query."""
    response = query_engine.query(query)
    return response.response

# Example usage for generating a response
original_query = "summary of event"
initial_response = generate_response(original_query, query_engine)

print("User Query:", original_query)
print("Initial AI Response:", initial_response)

# 2. Re-ranking
def rerank_results(nodes_with_scores, context, option):
    """Re-rank the results based on various scoring mechanisms."""

    # Create a list to hold nodes and their context scores
    scored_nodes = []

    if option == 1:  # Option 1: Re-rank based on the presence of context words
        context_words = set(context.lower().split())
        for node_with_score in nodes_with_scores:
            context_score = sum(1 for word in context_words if word in node_with_score.node.text.lower())
            scored_nodes.append((node_with_score, context_score))

    elif option == 2:  # Option 2: Re-rank based on original score only
        for node_with_score in nodes_with_scores:
            scored_nodes.append((node_with_score, 0))  # No context score, only use original score

    elif option == 3:  # Option 3: Re-rank based on length of text
        for node_with_score in nodes_with_scores:
            context_score = len(node_with_score.node.text.split())
            scored_nodes.append((node_with_score, context_score))

    elif option == 4:  # Option 4: Re-rank based on a combination of context score and original score
        context_words = set(context.lower().split())
        for node_with_score in nodes_with_scores:
            context_score = (node_with_score.score + sum(1 for word in context_words if word in node_with_score.node.text.lower())) / 2
            scored_nodes.append((node_with_score, context_score))

    elif option == 5:  # Option 5: Re-rank based on specific keywords
        keywords = set(["event", "success", "attendees"])  # Example keywords
        for node_with_score in nodes_with_scores:
            context_score = sum(1 for word in keywords if word in node_with_score.node.text.lower())
            scored_nodes.append((node_with_score, context_score))

    # Sort by context score and then by original score
    reranked_nodes = sorted(scored_nodes, key=lambda x: (x[1], x[0].score), reverse=True)

    return [node for node, _ in reranked_nodes]  # Return only the nodes

# Retrieve nodes with scores
query_string_1 = "summary of event?"
nodes_with_scores = retriever.retrieve(query_string_1)

# Iterate over each re-ranking option and generate a new response
for option in range(1, 6):  # Options 1 to 5
    reranked_nodes = rerank_results(nodes_with_scores, context, option)

    # Generate a new response based on the highest-ranked node
    if reranked_nodes:
        top_node = reranked_nodes[0]  # Get the highest-ranked node
        new_response = generate_response(top_node.node.text, query_engine)  # Generate response using the text of the top node

        # Display the new response first
        print(f"\nNew Response with Re-ranking Option {option}:")
        print(new_response)

        # Then display the top node and its score
        print(f"\nTop Node with Score for Re-ranking Option {option}:")
        print(f"Node Text: {top_node.node.text[:180]} ...")
        print(f"Score: {top_node.score:.4f}")

User Query: summary of event
Initial AI Response: The first side event, taking place in Large Auditorium 1 from 1:30 to 13:00, focuses on Language Models (LLMs) and Large Multimodal Models (LMMs). The goal of this session is to learn about and discuss the unique linguistic and cultural diversity associated with these models.

The second event is a Ministerial Council Meeting, which is a by-invitation-only gathering in the Summit Room.

The third side event occurs in Large Auditorium 1 from 14:30 to 16:00 and features case study presentations and a panel discussion. Technical experts will share their experiences related to creating real-world applications involving these models.

New Response with Re-ranking Option 1:
The convening will include a diverse group of experts representing various sectors such as government, international organizations, academia, industry, civil society, and other important stakeholders. This gathering aims to bring together different perspectives and context

In [ ]:
# 8. Initial Response Generation
def generate_response(query, query_engine):
    """Generate a response based on the user's query."""
    response = query_engine.query(query)
    return response.response

# Example usage
original_query = "summary of event"
initial_response = generate_response(original_query, query_engine)

print("User Query:", original_query)
print("Initial AI Response:", initial_response)

def collect_user_feedback(response, is_helpful):
    """Collect user feedback on the generated response."""
    # This function simulates collecting feedback from the user
    feedback = {
        'response': response,
        'is_helpful': is_helpful
    }
    # Store feedback for future analysis
    # In practice, you would save this to a database or file
    print("Feedback collected:", feedback)
    return feedback

# Prompt user for feedback on the initial response
user_feedback = input("Was the initial response helpful? (yes/no): ")
is_helpful = user_feedback.lower() == "yes"

# Collect user feedback
user_feedback = collect_user_feedback(initial_response, is_helpful)

User Query: summary of event
Initial AI Response: From 1:30 to 13:00, there is an event in Large Auditorium 1 focused on Language Models (LLMs) and Large Multimodal Models (LMMs). This session aims to facilitate learning and sharing of knowledge about the unique linguistic and cultural diversity associated with these models.

Later, from 14:30 to 16:00, there is another event in the Summit Room, which is a Ministerial Council Meeting. This meeting is by invitation only.

Lastly, from 14:30 to 16:00, there is a Side Event 3 in Auditorium 1. This event consists of case study presentations and a panel discussion, where technical experts share their experiences of working in the real world.
Was the initial response helpful? (yes/no): no
Feedback collected: {'response': 'From 1:30 to 13:00, there is an event in Large Auditorium 1 focused on Language Models (LLMs) and Large Multimodal Models (LMMs). This session aims to facilitate learning and sharing of knowledge about the unique linguistic

In [ ]:
# 9. Incorporating Feedback into Prompt with Context
def generate_response_with_feedback(query, original_response, feedback):
    """Generate a response using the original query, AI's response, and user feedback."""
    # Combine the query, original response, and feedback into a prompt
    prompt = (
        f"User query: {query}\n"
        f"Original AI response: {original_response}\n"
        f"User feedback: {'Helpful' if feedback['is_helpful'] else 'Not Helpful'}\n"
        f"Based on the query and feedback, generate a more helpful response:"
    )

    # Use the prompt to generate a new response
    new_response = query_engine.query(prompt)

    return new_response.response

# Generate a new response using the original query, initial response, and feedback
new_response = generate_response_with_feedback(original_query, initial_response, user_feedback)

print("\nUser Feedback:", "Helpful" if user_feedback['is_helpful'] else "Not Helpful")
print("\nNew Response (with Feedback):")
print(new_response)


User Feedback: Not Helpful

New Response (with Feedback):
The event from 1:30 to 13:00, held in Large Auditorium 1, is dedicated to exploring Language Models (LLMs) and Large Multimodal Models (LMMs). This session offers an opportunity to learn about and discuss the unique linguistic and cultural aspects related to these models.

In the afternoon, there are two concurrent events. The first one, taking place in a designated room, is a Ministerial Council Meeting. This meeting is exclusive and open only by invitation.

The second event, happening in Auditorium 1, is Side Event 3. This session features case study presentations and a panel discussion, allowing technical experts to share their practical experiences in the field.


In [ ]:
# 10. Query Expansion
from sklearn.feature_extraction.text import TfidfVectorizer

def expand_query(original_query, documents, num_expansions=3):
    """Expand the query using related terms from the document set."""
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Transform the original query into a vector
    query_vector = vectorizer.transform([original_query])

    # Calculate cosine similarity between the query and documents
    cosine_similarities = (tfidf_matrix * query_vector.T).toarray().flatten()

    # Get indices of the top N most similar documents
    top_indices = cosine_similarities.argsort()[-num_expansions:][::-1]

    # Extract the top terms from the most similar documents
    related_terms = set()
    for index in top_indices:
        terms = vectorizer.get_feature_names_out()
        # Get the terms from the document
        document_terms = tfidf_matrix[index].toarray().flatten()
        # Add the top terms to the related terms set
        for term, score in zip(terms, document_terms):
            if score > 0.1:  # Only include terms with a significant score
                related_terms.add(term)

    # Combine original query with related terms
    expanded_query = f"{original_query} {' '.join(related_terms)}"
    return expanded_query

# Example documents
documents = [
    "The event was a great success with many attendees.",
    "Attendees enjoyed various activities and networking opportunities.",
    "The conference covered a range of topics relevant to the industry."
]

# Original query
original_query = "summary of event"

# Expand the query
expanded_query = expand_query(original_query, documents)

# Print the original and expanded queries
print("Original Query:", original_query)
print("Expanded Query:", expanded_query)

# Generate responses for both queries
original_response = generate_response(original_query, query_engine)
expanded_response = generate_response(expanded_query, query_engine)

# Print the responses
print("\nResponse for Original Query:")
print(original_response)
print("\nResponse for Expanded Query:")
print(expanded_response)

# Instructions for Students:
# - To explore different queries, change the `original_query` variable to a new query.
# - Edit the `documents` list to include content that is relevant to the new query.
# - Rerun the cell to see how the expanded query and responses change based on the new input.

Original Query: summary of event
Expanded Query: summary of event great opportunities activities various covered relevant topics attendees success conference enjoyed industry event range networking

Response for Original Query:
The events scheduled are focused on knowledge sharing and learning. The first side event, taking place in Large Auditorium 1 from 1:30 to 13:00, is centered around Language Models (LLMs) and Large Multimodal Models (LMMs), aiming to explore their unique linguistic and cultural diversity. The second event is a Ministerial Council Meeting, which is by invitation only and will be held in the Summit Room. The final side event, from 14:30 to 16:00 in Large Auditorium 1, consists of case study presentations and a panel discussion, bringing together technical experts to share their experiences in real-world settings.

Response for Expanded Query:
The event provided great opportunities for attendees to explore various relevant topics in the AI industry. The activities i